In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1)

## nn.Module 을 사용한 선형 회귀 구현

### 단순 선형 회귀 구현

$y = 2x$ 를 가정한 데이터를 예시로 보면, W=2, b=0인 모델을 찾는 과정

model = nn.Linear(input_dim, output_dim)

입력과 출력 차원 모두 단순 선형 회귀 이므로 1이다.

In [2]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [3]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)

In [6]:
# model에는 가중치 W와 편향 b가 저장되어 있고 .parameters()로 불러올 수 있다
print(list(model.parameters()))
# 학습하지 않은 상태이므로 랜덤한 값으로 초기화되어 있다.
# 두 값 모두 학습의 대상이므로 requires_grad=True 이다.

[Parameter containing:
tensor([[-0.8612]], requires_grad=True), Parameter containing:
tensor([0.3583], requires_grad=True)]


In [7]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

In [8]:
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print(f'Epoch {epoch:4d}/{nb_epochs} \
             Cost: {cost.item():.6f}')

Epoch    0/2000              Cost: 34.230083
Epoch  100/2000              Cost: 0.162711
Epoch  200/2000              Cost: 0.100546
Epoch  300/2000              Cost: 0.062131
Epoch  400/2000              Cost: 0.038393
Epoch  500/2000              Cost: 0.023725
Epoch  600/2000              Cost: 0.014660
Epoch  700/2000              Cost: 0.009059
Epoch  800/2000              Cost: 0.005598
Epoch  900/2000              Cost: 0.003459
Epoch 1000/2000              Cost: 0.002138
Epoch 1100/2000              Cost: 0.001321
Epoch 1200/2000              Cost: 0.000816
Epoch 1300/2000              Cost: 0.000504
Epoch 1400/2000              Cost: 0.000312
Epoch 1500/2000              Cost: 0.000193
Epoch 1600/2000              Cost: 0.000119
Epoch 1700/2000              Cost: 0.000074
Epoch 1800/2000              Cost: 0.000045
Epoch 1900/2000              Cost: 0.000028
Epoch 2000/2000              Cost: 0.000017


In [11]:
# 학습된 모델을 확인하기 위해 x에 임의의 값 4를 넣어 모델이 예측하는 y값을 확인

# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 

# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
# H(x)식에 입력x로부터 예측된 값을 얻는 것을 forward 연산이라 함
pred_y = model(new_var) # forward 연산

# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print(pred_y)

# 가중치 W와 편향b는 2와 0에 가까워야 함
# 학습 후 W, b 출력
print(list(model.parameters()))

tensor([[7.9917]], grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[1.9952]], requires_grad=True), Parameter containing:
tensor([0.0110], requires_grad=True)]


`forward` : $H(x)$식에 입력 $x$로부터 예측된 $y$를 얻는 연산
- prediction = model(x_train) : x_train으로부터 예측값을 리턴
- pred_y = model(new_var) : new_var로부터 예측값을 리턴
  
`backward` : 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 연산
- cost.backward() : 비용 함수로부터 기울기를 구함

### 다중 선형 회귀 구현

$ H(X) = w_1x_1 + w_2x_2 + w_3x_3 $ 를 구현, 입력이 3이고 출력은 1

In [12]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [14]:
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)
# 3개의 가중치와 하나의 편향을 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.4863, -0.1562,  0.0478]], requires_grad=True), Parameter containing:
tensor([0.0566], requires_grad=True)]


In [15]:
# optimizer 설정. 학습률(learning rate)은 0.00001로 설정, 학습률이 0.01이면 기울기가 발산함
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [16]:
# 단순 선형 회귀를 구현과 동일하게 구현
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print(f'Epoch {epoch:4d}/{nb_epochs} \
             Cost: {cost.item():.6f}')

Epoch    0/2000              Cost: 19560.238281
Epoch  100/2000              Cost: 0.379555
Epoch  200/2000              Cost: 0.370078
Epoch  300/2000              Cost: 0.361095
Epoch  400/2000              Cost: 0.352587
Epoch  500/2000              Cost: 0.344504
Epoch  600/2000              Cost: 0.336844
Epoch  700/2000              Cost: 0.329577
Epoch  800/2000              Cost: 0.322684
Epoch  900/2000              Cost: 0.316156
Epoch 1000/2000              Cost: 0.309954
Epoch 1100/2000              Cost: 0.304076
Epoch 1200/2000              Cost: 0.298492
Epoch 1300/2000              Cost: 0.293202
Epoch 1400/2000              Cost: 0.288175
Epoch 1500/2000              Cost: 0.283415
Epoch 1600/2000              Cost: 0.278886
Epoch 1700/2000              Cost: 0.274593
Epoch 1800/2000              Cost: 0.270514
Epoch 1900/2000              Cost: 0.266645
Epoch 2000/2000              Cost: 0.262971


In [18]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
# 73, 80, 75는 훈련데이터 값으로 y가 152에 가까워야 함
print(pred_y)
print(list(model.parameters()))

tensor([[151.0054]], grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[1.0030, 0.4258, 0.5821]], requires_grad=True), Parameter containing:
tensor([0.0641], requires_grad=True)]
